<a href="https://colab.research.google.com/github/ugurcankuzu/XAI/blob/main/omnixai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu notebookta bir diğer XAI kütüphanesi olan omniXAI'a ait örnek kullanıma göz atacağız.

Öncelikle gerekli pip kurulumunu yapıp projemize dahil edelim.

In [ ]:
!pip install omnixai
import omnixai
from omnixai.explainers.tabular import TabularExplainer
from omnixai.data.tabular import Tabular
from omnixai.preprocessing.tabular import TabularTransform
from omnixai.preprocessing.base import Identity
import pickle

Daha önceden paketlediğimiz verilerimiz ve metodumuzu proje içerisinde pickle ile tanımlayalım.

In [ ]:
with open("datasetAndModelOut.pkl","rb") as f:
  lr = pickle.load(f)
  testX = pickle.load(f)
  trainX = pickle.load(f)
  testY = pickle.load(f)
  trainY = pickle.load(f)

Elimizdeki model lineer olduğu için bunu açıklayacak explainer'ımız bizden yollanacak verilerin Tabular olmasını beklemektedir. Bu yüzden önceden paketlediğimiz ve dahil ettiğimiz testX verilerimizi Tabular hale getirelim ve dönüştürelim.

In [ ]:
testXTabular = Tabular(testX.to_numpy())
transformer = TabularTransform(Identity()).fit(testXTabular)
x = transformer.transform(testXTabular)

Tabular hale getirdiğimiz verileri oluşturacağımız TabularExplainer içerisine argüman olarak verelim. Bunun yanı sıra diğer argümanlarımız ise açıklamanın hangi metodlarca yapılacağını içeren bir array, tip olarak regression, ve önişleme için yazdığımız lambda fonksiyondur.

In [ ]:
preprocess = lambda z: transformer.transform(z)
omniExp = TabularExplainer(["shap","lime"],"regression",testXTabular,lr,preprocess)

X does not have valid feature names, but LinearRegression was fitted with feature names


Kurduğumuz tabular explainer'ı kullanarak artık local ve global açıklamalar elde edebiliriz.

 Aşağıdaki kod bloğunda local için testX içerisinden 5. indexteki örneği ele aldık. 


Global açıklama ise tüm veri üzerinde ağırlıkların incelenmesini sağlamaktadır.

In [ ]:
local = omniExp.explain(testXTabular[:5])
globalExp = omniExp.explain_global()

Local açıklama için Tabular Explainer'a verdiğimiz açıklama metodlarını da kullanarak çizimleri yaptırabiliriz. Aşağıda Shap ve Lime kullanarak yapılan açıklamaları görebilirsiniz. Yine bu grafiğe bakarak modelimizin tahminindeki en etken unsurları görebiliriz. Diğer kütüphanelerde olduğu gibi en çok present_price unsurunun etkili olduğu görülmektedir.

In [ ]:
local["shap"].ipython_plot(0)


In [ ]:
local["lime"].ipython_plot(0)